RANDOM FOREST CLASSIFIER

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV

In [2]:
#dataset
df = pd.read_csv('dataset.csv')

label_encoder = LabelEncoder()
rf = RandomForestClassifier()

REMOVE UNKNOWNS FROM TARGETS VS FROM ALL COLUMNS (RUN ONE OF THE CELLS)

Without unknown targets (possible to apply just to one of the columns if only one of them is the target)

In [3]:
# Replace 'Unknown' with NaN
df['Genetic Disorder'].replace('Unknown', np.nan, inplace=True)
df['Disorder Subclass'].replace('Unknown', np.nan, inplace=True)

df.dropna(inplace=True)

Without unknown in any column

In [ ]:
# Replace 'Unknown' with NaN
df.replace('Unknown', np.nan, inplace=True)

df.dropna(inplace=True)

USE JUST LABEL ENCODER OR BOTH LABE LENCODER AND ONE-HOT ENCODER (RUN ONE OF THE CELLS)

Just label encoder

In [ ]:
all_x_columns= [
    'Patient Age', "Genes in mother's side", 'Inherited from father', 'Maternal gene', 'Paternal gene',
    'Blood cell count (mcL)', "Mother's age", "Father's age", 'Status', 'Respiratory Rate (breaths/min)',
    'Heart Rate (rates/min)', 'Follow-up', 'Gender', 'Birth asphyxia',
    'Autopsy shows birth defect (if applicable)', 'Place of birth', 'Folic acid details (peri-conceptional)',
    'H/O serious maternal illness', 'H/O radiation exposure (x-ray)', 'H/O substance abuse',
    'Assisted conception IVF/ART', 'History of anomalies in previous pregnancies', 'No. of previous abortion',
    'Birth defects', 'White Blood cell count (thousand per microliter)', 'Blood test result',
    'Symptom 1', 'Symptom 2', 'Symptom 3', 'Symptom 4', 'Symptom 5'
]

df_encoded=df.copy()
for column in all_x_columns:
    df_encoded[column] = label_encoder.fit_transform(df_encoded[column].astype(str))

Label encoder and one-hot encoder

In [ ]:
categorical_unordered_columns = [
    'Genes in mother\'s side', 'Inherited from father', 'Maternal gene', 'Paternal gene',
    'Gender', 'Birth asphyxia',
    'Autopsy shows birth defect (if applicable)', 'Place of birth',
    'Folic acid details (peri-conceptional)', 'H/O serious maternal illness',
    'H/O radiation exposure (x-ray)', 'H/O substance abuse', 'Status',
    'Assisted conception IVF/ART', 'History of anomalies in previous pregnancies',
    'Symptom 1', 'Symptom 2', 'Symptom 3', 'Symptom 4', 'Symptom 5'
]

quantitative_with_unknowns_or_ordered_columns = ['Patient Age', "Mother's age", "Father's age", 'Respiratory Rate (breaths/min)',
    'Heart Rate (rates/min)', 'Follow-up', 'No. of previous abortion',
    'Birth defects', 'White Blood cell count (thousand per microliter)', 'Blood test result']


df_encoded=df.copy()
for column in quantitative_with_unknowns_or_ordered_columns:
    df_encoded[column] = label_encoder.fit_transform(df_encoded[column].astype(str))
df_encoded=pd.get_dummies(df_encoded, columns=categorical_unordered_columns, drop_first=False)

With Feature Selection

In [4]:
# With feature selection

df=df[["Symptom 5", "Symptom 4", "Symptom 3", "Symptom 2", "Genes in mother's side", "Inherited from father","Genetic Disorder","Disorder Subclass"]]

df_encoded=df.copy()
df_encoded=pd.get_dummies(df_encoded, columns=["Symptom 5", "Symptom 4", "Symptom 3", "Symptom 2", "Genes in mother's side", "Inherited from father"], drop_first=False)

TRAIN MODEL TO PREDICT GENETIC DISORDER VS DISORDER SUBCLASS (RUN ONE OF THE CELLS)

Genetic Disorder

In [ ]:
# Extract features and target
X = df_encoded.drop(columns=['Genetic Disorder', 'Disorder Subclass'], axis=1)
y = df_encoded['Genetic Disorder']

Disorder subclass

In [ ]:
# Extract features and target
X = df_encoded.drop(columns=['Genetic Disorder', 'Disorder Subclass'], axis=1)
y = df_encoded['Disorder Subclass']

Both

In [5]:
# Extract features and target
X = df_encoded.drop(columns=['Genetic Disorder', 'Disorder Subclass'], axis=1)
y = df_encoded[['Genetic Disorder','Disorder Subclass']]

SCALE FEATURES (POSSIBLE TO SKIP)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

BALANCE DATA - ONE TARGET (POSSIBLE TO SKIP)

In [ ]:
class_counts = y.value_counts()

# Calculate class weights
total_samples = len(y)
class_weights = {label: 1/count for label, count in class_counts.items()}

# Classifier with weights
rf = RandomForestClassifier(class_weight=class_weights)

BALANCE DATA - MULTITARGET (POSSIBLE TO SKIP)

In [6]:
# GENETIC DISORDER
class_counts_gd = y['Genetic Disorder'].value_counts()

# Calculate class weights
total_samples_gd = len(y['Genetic Disorder'])
class_weights_gd = {label: 1/count for label, count in class_counts_gd.items()}


# DISORDER SUBCLASS
class_counts_ds = y['Disorder Subclass'].value_counts()

# Calculate class weights
total_samples_ds = len(y['Disorder Subclass'])
class_weights_ds = {label: 1/count for label, count in class_counts_ds.items()}


class_weights=[class_weights_gd, class_weights_ds]

# Classifier with weights
rf = RandomForestClassifier(class_weight=class_weights)

TUNING (POSSIBLE TO SKIP)

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
parameters = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:

rf = RandomizedSearchCV(estimator = rf, param_distributions = parameters, n_iter = 100, cv = 3, verbose=2, refit=True, scoring='balanced_accuracy')

TRAIN MODEL

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, stratify=y) # training as 50%
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test) 


Genetic Disorder
                                              precision    recall  f1-score   support

 Mitochondrial genetic inheritance disorders       0.69      0.38      0.49      2311
Multifactorial genetic inheritance disorders       0.21      0.88      0.33       469
            Single-gene inheritance diseases       0.43      0.31      0.36      1732

                                    accuracy                           0.40      4512
                                   macro avg       0.44      0.52      0.39      4512
                                weighted avg       0.54      0.40      0.42      4512


Balanced accuracy:
0.521127156341401

Macro f1:
0.3931028418578197

Weighted f1:
0.4219809772515377

Disorder Subclass
                                     precision    recall  f1-score   support

                        Alzheimer's       0.10      0.85      0.18        33
                             Cancer       0.05      0.78      0.09        23
                    Cysti

PRINT SCORES MULTIOUTPUT VS ONE TARFET (RUN ONE OF THE CELLS)

Multioutput

In [ ]:
print('\nGenetic Disorder')
print(classification_report(y_test.iloc[:,-2], y_pred[:,-2]))
print('\nBalanced accuracy:')
print(balanced_accuracy_score(y_test.iloc[:,-2], y_pred[:,-2]))
print('\nMacro f1:')
print(f1_score(y_test.iloc[:,-2], y_pred[:,-2], average='macro'))
print('\nWeighted f1:')
print(f1_score(y_test.iloc[:,-2], y_pred[:,-2], average='weighted'))

print('\nDisorder Subclass')
print(classification_report(y_test.iloc[:,-1], y_pred[:,-1]))
print('\nBalanced accuracy:')
print(balanced_accuracy_score(y_test.iloc[:,-1], y_pred[:,-1]))
print('\nMacro f1:')
print(f1_score(y_test.iloc[:,-1], y_pred[:,-1], average='macro'))
print('\nWeighted f1:')
print(f1_score(y_test.iloc[:,-1], y_pred[:,-1], average='weighted'))

One target

In [ ]:
print('\nGenetic Disorder')
print(classification_report(y_test, y_pred))
print('\nBalanced accuracy:')
print(balanced_accuracy_score(y_test, y_pred))
print('\nMacro f1:')
print(f1_score(y_test, y_pred, average='macro'))
print('\nWeighted f1:')
print(f1_score(y_test, y_pred, average='weighted'))